## Bibliotecas necessárias

```
pip install pandas
pip install numpy
pip scikit-learn
```

In [52]:
import pandas as pd
import numpy as np
import re

from scipy.stats import zscore

In [53]:
DATASET_FOLDER_PATH = './datasets/'

def read_csv(name):
	return pd.read_csv(DATASET_FOLDER_PATH + name + '.csv')

## Carregamento dos datasets necessários
	- execucoes.csv
	- atividades.csv
	- solucoes.csv

In [54]:
attempts = read_csv('execucoes')
activities = read_csv('atividades')
solutions = read_csv('solucoes')
exams = activities.query('tipo == "exam"')

## Criando um DataFrame de tentativas de exames presenciais

### Critérios

	- A questão tem ao menos 20 tentativas de estudantes diferentes
	- A tentativa tem ao menos uma submissão
	- Tem campo tempo foco não nulo

In [55]:
def min_attempts_filter(exam_attempts, min_attempts = 20):
	'Cria uma lista de códigos de questões que tem ao menos o mínimo de tentativas'

	grouped_attempts = pd.DataFrame()
	exam_attempts_exercicio = exam_attempts.groupby('exercicio')['exercicio'].count()
	grouped_attempts['exercicio'] = exam_attempts_exercicio.index
	grouped_attempts['attempts'] = exam_attempts_exercicio.values
	selection = grouped_attempts.attempts >= min_attempts

	return grouped_attempts[selection].exercicio

In [56]:
def filter_outliers(df, col):
	selection = np.abs(zscore(df[col]) < 3)
	return df[selection]

In [57]:
def get_implementation_time_in_seconds(raw_time):
	'Converte o campo de tempo de implementação para segundos'
	
	i_time = pd.to_timedelta(raw_time.str.strip())
	i_time = i_time.apply(lambda x: (x.total_seconds()))

	return i_time


In [58]:
def get_exam_attempts():
	'Filtra tentativas validas de exames presenciais'
	
	exam_codes = exams.codigo.unique()
	exam_attempts = attempts.loc[attempts.atividade.isin(exam_codes)]

	print(f"Questoes pre-filtro {len(exam_attempts.groupby('exercicio'))}")

	exam_attempts = exam_attempts.query('n_submissoes != 0')
	exam_attempts = exam_attempts.dropna(subset=['tempo_foco'])
	
	exam_attempts.acertou = exam_attempts.nota_final > 99.99	
	min_attempts_code = min_attempts_filter(exam_attempts)
	exam_attempts = exam_attempts[exam_attempts.exercicio.isin(min_attempts_code)]

	exam_attempts['tempo_foco'] = get_implementation_time_in_seconds(exam_attempts['tempo_foco'])
	exam_attempts = filter_outliers(exam_attempts, 'tempo_foco')

	return exam_attempts

## Derivando features booleanas

In [59]:
def add_boolean_features(df):
	df['has_operators'] = df['h1'] > 0
	df['has_operands'] = df['h2'] > 0
	df['has_imports'] = df['imports'] > 0
	df['has_kwds'] = df['kwds_unique'] > 0
	df['has_numbers'] = df['lt_numbers'] > 0
	df['has_strs'] = df['lt_strings'] > 0
	df['has_bools'] = df['lt_booleans'] > 0
	df['has_lgc_op'] = df['lgc_op_unique'] > 0
	df['has_arth_op'] = df['arithmetic_op_unique'] > 0
	df['has_cmp_op'] = df['cmp_op_unique'] > 0
	df['has_conditionals'] = df['conditionals'] > 0
	df['has_ifs'] = df['ifs'] > 0
	df['has_elifs'] = df['elifs'] > 0
	df['has_elses'] = df['elses'] > 0
	df['has_loops'] = df['loops'] > 0
	df['has_fors'] = df['fors'] > 0
	df['has_whiles'] = df['whiles'] > 0
	df['has_add_sub'] = (df['add_op'] + df['minus_op']) > 0
	df['has_mult_div'] = (df['mult_op'] + df['div_op'] + df['div_floor_op']) > 0
	df['has_blocks'] = df['colons'] > 0

## Criando um DataFrame agrupando dados das tentativas dos estundantes

Esse dataframe contém a média do número de tentativas e do tempo de implementação.

In [60]:
def create_exercise_stat_df(filtered_attempts):
	exercise_data = filtered_attempts.groupby('exercicio')
	df = pd.DataFrame()
	
	df['n_attempts'] = exercise_data.n_submissoes.mean() + exercise_data.n_testes.mean()
	df['tempo_foco'] = exercise_data.tempo_foco.mean()

	return df

## Recupenrando o modulo de cada questão

O modulo será utlizado para separar as questões por assunto durante a classificação.

In [61]:
def format_titulo(titulo: str):
    titulo = re.sub(r'[\(\)\.\-\\:–\/]', ' ', titulo.lower())
    titulo = re.sub(r'[áàã]', 'a', titulo)
    titulo = re.sub(r'[éèẽ]', 'e', titulo)
    titulo = re.sub(r'[íìĩ]', 'i', titulo)
    titulo = re.sub(r'[óòõ]', 'o', titulo)
    titulo = re.sub(r'[úùũ]', 'u', titulo)
    titulo = re.sub(r'ç', 'c', titulo)
    titulo = re.sub(r'\sde\s', ' ', titulo)
    titulo = re.sub(r'\scod\s|\scodificacao\s', ' ', titulo)
    titulo = re.sub(r'\sest\s', ' estrutura ', titulo)
    titulo = re.sub(r'^tp|^lab\s|^lab01\s|^lab0\s|^laboratorio\s|^exercicios desafio\s|^desafios\s|^desafio\s', ' trabalho pratico ', titulo)
    titulo = re.sub(r'\s0\s', ' 00 ', titulo)
    titulo = re.sub(r'\s+', ' ', titulo)
    titulo = titulo.strip()
    titulo = re.sub('^trabalho pratico ', 'tp ', titulo)
    titulo = re.sub('^t ', 'tp ', titulo)
    titulo = re.sub(r'\s00\s', ' 0 ', titulo)
    titulo = re.sub(r'\s01\s', ' 1 ', titulo)
    titulo = re.sub(r'\s02\s', ' 2 ', titulo)
    titulo = re.sub(r'\s03\s', ' 3 ', titulo)
    titulo = re.sub(r'\s04\s', ' 4 ', titulo)
    titulo = re.sub(r'\s05\s', ' 5 ', titulo)
    titulo = re.sub(r'\s06\s', ' 6 ', titulo)
    titulo = re.sub(r'\s07\s', ' 7 ', titulo)
    
    if titulo.startswith('tp 0') or titulo.startswith('tp p'):
        return 'M01' # "tp 0" também envolve variáveis e estrutura sequencial
    elif titulo.startswith('tp 1') or titulo.startswith('tp v'):
        return 'M01'
    elif titulo.startswith('tp 2'):
        return 'M02'
    elif titulo.startswith('tp 3'):
        return 'M03'
    elif titulo.startswith('tp 4'):
        return 'M04'
    elif titulo.startswith('tp 5'):
        return 'M05'
    elif titulo.startswith('tp 6'):
        return 'M06'
    elif titulo.startswith('tp 7'):
        return 'M07'
    elif titulo.startswith('prova'):
        return 'PF'
    elif titulo.startswith('tp s') or titulo.startswith('tp r'):
        return 'EXTRA'

    return None

def get_activity_module_series():
    exams_copy = exams.copy()
    exams_copy.titulo = exams_copy.titulo.apply(format_titulo)

    not_tp = ['EXTRA', 'PF']
    trabalhos_praticos = exams_copy.loc[~exams_copy.titulo.isin(not_tp)]

    modulo_questao = dict()

    for idx in trabalhos_praticos.index:
        titulo = trabalhos_praticos.loc[idx, 'titulo'] # recupera o titulo da atividade
        blocos = trabalhos_praticos.loc[idx, 'blocos'] # recupera o bloco de questões da atividade
        blocos = blocos.replace('[','').replace(']','').replace(' ','').split(',') # separa os códigos das questões que estão no bloco da ativida
        for questao in blocos: # adiciona o módulo da questão (titulo da atividade) no dicionário
            modulo_questao[int(questao)] = titulo

    modulo_questao = pd.Series(modulo_questao)
    return modulo_questao

def add_activity_modules_col(df):
    modulo_questao = get_activity_module_series()
    df['module'] = modulo_questao

## Criando o DataFrame final para os experimentos

Os dados finais incluem os dados agrupados das tentativas, bem como os dados da solução.

Os dados da solução do professor serão utilizados com variáveis independentes, e os dados das tentativas como variáveis dependentes.

## Complementando o dataset de soluções

Nem todas as questões do dataset de questões possuem uma solução fornecida
pelo professor.

Por isso, foi necessário selecionar algumas soluções dos alunos para complementar o dataset.

O critério de seleção é o menor tempo de implementação (campo tempo_foco).

Duas restrições foram estabelecidadas:
	- O aluno deve ter acertado a questão
	- O tempo de solução deve ser no mínimo 1 minuto

In [62]:
def get_exercises_that_dont_have_a_solution(exercise_stats, solutions):
	missing = list(exercise_stats[exercise_stats.join(solutions.set_index('codigo')).h1.isnull()].index)
	return missing

def get_solution_candidates(missing_codes, exam_attempts):
	return exam_attempts[
		(exam_attempts.exercicio.isin(missing_codes)) &
		(exam_attempts.acertou) &
		(exam_attempts.tempo_foco > 60)
	]

def create_missing_solutions_df(selected_solutions, solutions_raw):
	selected_solutions['codigo'] = selected_solutions.index
	columnsToBeRemoved = set(selected_solutions.columns) - set(solutions_raw.columns)
	selected_solutions.drop(columns=columnsToBeRemoved, inplace=True)

	return selected_solutions

def get_solutions_df(exam_attempts, exercise_stats, solutions_raw):
	missing = get_exercises_that_dont_have_a_solution(exercise_stats, solutions_raw)
	solution_candidates = get_solution_candidates(missing, exam_attempts)

	selected_solutions = solution_candidates.groupby('exercicio').min()
	selected_solutions = create_missing_solutions_df(selected_solutions, solutions_raw)

	return pd.concat((selected_solutions, solutions_raw))

In [63]:
def create_final_df(exercise_stats, solutions):
	data = exercise_stats.join(solutions.set_index('codigo')).dropna()
	add_boolean_features(data)
	add_activity_modules_col(data)

	return data

In [64]:
def get_data():
	exam_attempts = get_exam_attempts()
	exercise_stats = create_exercise_stat_df(exam_attempts)
	final_solutions = get_solutions_df(exam_attempts, exercise_stats, solutions)
	data = create_final_df(exercise_stats, final_solutions)
	data.dropna(subset=['module'], inplace=True)
		
	return data

## Salvando o resultado final

In [65]:
data = get_data()
print(f"Questoes pos-filtro {len(data)}")
data.to_csv('./final_df.csv', index=False)

Questoes pre-filtro 656
Questoes pos-filtro 328
